In [1]:
import h5py
import yaml
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys; sys.path.insert(0, '..')
#sys.path.append('/dfs6/pub/nsankar1/pushes_bubbleML/BubbleML/sciml/models/ConvolutionalNeuralOperator')
from sciml.models.trunk_wrapper.trunk_wrapper import TrunkWrapper
from sciml.models.pdearena.unet import Unet
from sciml.op_lib.disk_hdf5_dataset import DiskTempVelDataset
from torch.utils.data import ConcatDataset, Dataset, DataLoader
from sciml.op_lib.disk_hdf5_dataset import (
        DiskTempInputDataset,
        DiskTempVelDataset
)
from sciml.op_lib.hdf5_dataset import (
        HDF5ConcatDataset,
        TempInputDataset,
        TempVelDataset
)

DEVICE = 'cpu'

In [6]:
def collate_fn(data):
        fw = 5       
        inbatch = []
        labelbatch = []
        
        for pack in data:
            coords, temp, vel, dfun, temp_label, vel_label = pack
            input = (coords, temp, vel, dfun)
            
            for t in range(fw):
                inbatch += [input + (torch.tensor(t, device=data[0][0].device).unsqueeze(0),)]
                labelbatch += [(temp_label[:, t].unsqueeze(1), vel_label[:, t*2:(t+1)*2])]
        del pack
        return tuple(list(map(torch.cat, zip(*inbatch)))+list(map(torch.cat, zip(*labelbatch))))

In [8]:
checkpoint_path = "/share/crsp/lab/amowli/hthomare/BubbleML/outputs/TrunkWrapper_vel_dataset_250_1710180567.pt"
experiment_yaml_file = "/share/crsp/lab/amowli/hthomare/BubbleML/conf/experiment/experimental/unet_arena/pb_vel.yaml"

with open(experiment_yaml_file, 'r') as file:
    model_params = yaml.safe_load(file)

checkpoint = torch.load(checkpoint_path)
print(checkpoint.keys())

val_file = 'Twall-103.hdf5'
val_dataset = TempVelDataset(val_file, 0, True, False, 5, 5, 1) 
val_dataset.normalize_temp_()
val_dataset.normalize_vel_()
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)


# model = Unet(
#             in_channels=17,
#              out_channels=15,
#              hidden_channels=model_params['model']['hidden_channels'],
#              ch_mults=[1,2,2,4,4],
#              is_attn=[False]*5,
#              activation= 'gelu',
#             #  mid_attn=False,
#              norm=True,
#              use1x1=True
#             )



RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
for image, _ in val_dataloader: 
    print(image.dim)
# plt.plot(image[0].cpu().numpy())

NameError: name 'cfg' is not defined

In [ ]:
Trunk = TrunkWrapper(model, 5, 5, val_dataloader.datum_dim())

checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
print(checkpoint)

In [ ]:
input, label = val_dataset[35]
input = input.to(DEVICE).float().unsqueeze(0)
label = label.to(DEVICE).float()

trunk_val = torch.range(1, 6, requires_grad=True)

pred = model(input, trunk_val)

label = label.squeeze().cpu().numpy()
pred = pred.squeeze().detach().cpu().numpy()
abs_err = np.abs(label - pred)

fig, ax = plt.subplots(1, 3, figsize=(10, 5))

data = {
    'Ground Truth': label,
    'Predicted': pred,
    'Abs. Error': abs_err
}

for idx, (key, im) in enumerate(data.items()):
    im = ax[idx].imshow(np.flipud(im))
    fig.colorbar(im, ax=ax[idx], shrink=0.45)
    ax[idx].set_title(key)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])